In [1]:
import pyspark
from pyspark.sql import SparkSession
import pandas as pd
from pyspark.sql import types

In [2]:
spark = SparkSession.builder \
    .master("local[*]") \
    .appName('test') \
    .getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/03/11 05:51:31 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
25/03/11 05:51:32 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


### Read data

In [3]:
df_yellow = spark.read.parquet('../data/raw/yellow/2024/10/yellow_tripdata_2024-10.parquet')

In [4]:
df_yellow.printSchema()

root
 |-- VendorID: integer (nullable = true)
 |-- tpep_pickup_datetime: timestamp_ntz (nullable = true)
 |-- tpep_dropoff_datetime: timestamp_ntz (nullable = true)
 |-- passenger_count: long (nullable = true)
 |-- trip_distance: double (nullable = true)
 |-- RatecodeID: long (nullable = true)
 |-- store_and_fwd_flag: string (nullable = true)
 |-- PULocationID: integer (nullable = true)
 |-- DOLocationID: integer (nullable = true)
 |-- payment_type: long (nullable = true)
 |-- fare_amount: double (nullable = true)
 |-- extra: double (nullable = true)
 |-- mta_tax: double (nullable = true)
 |-- tip_amount: double (nullable = true)
 |-- tolls_amount: double (nullable = true)
 |-- improvement_surcharge: double (nullable = true)
 |-- total_amount: double (nullable = true)
 |-- congestion_surcharge: double (nullable = true)
 |-- Airport_fee: double (nullable = true)



### Partition data and write it to storage

In [6]:
year = 2024
month = 10
print(f'processing data for {year}/{month}')

input_path = f'../data/raw/yellow/{year}/{month:02d}/yellow_tripdata_{year}-{month:02d}.parquet'
output_path = f'../data/pq/yellow/{year}/{month:02d}/'

df_yellow = spark.read.parquet(input_path)
df_yellow.printSchema()

df_yellow \
    .repartition(4) \
    .write.parquet(output_path, mode='overwrite')

processing data for 2024/10
root
 |-- VendorID: integer (nullable = true)
 |-- tpep_pickup_datetime: timestamp_ntz (nullable = true)
 |-- tpep_dropoff_datetime: timestamp_ntz (nullable = true)
 |-- passenger_count: long (nullable = true)
 |-- trip_distance: double (nullable = true)
 |-- RatecodeID: long (nullable = true)
 |-- store_and_fwd_flag: string (nullable = true)
 |-- PULocationID: integer (nullable = true)
 |-- DOLocationID: integer (nullable = true)
 |-- payment_type: long (nullable = true)
 |-- fare_amount: double (nullable = true)
 |-- extra: double (nullable = true)
 |-- mta_tax: double (nullable = true)
 |-- tip_amount: double (nullable = true)
 |-- tolls_amount: double (nullable = true)
 |-- improvement_surcharge: double (nullable = true)
 |-- total_amount: double (nullable = true)
 |-- congestion_surcharge: double (nullable = true)
 |-- Airport_fee: double (nullable = true)



# Use SQL with data

In [7]:
df_yellow.createOrReplaceTempView('yellow_trips_data')

In [8]:
spark.sql("""
SELECT
    count(1)
FROM
    yellow_trips_data
WHERE
    tpep_pickup_datetime >= '2024-10-15' AND
    tpep_pickup_datetime < '2024-10-16'    
""").show()

+--------+
|count(1)|
+--------+
|  128893|
+--------+



# Longest distance

In [31]:
spark.sql("""
SELECT
    trip_distance,
    DATEDIFF(hour, tpep_pickup_datetime, tpep_dropoff_datetime) as trip_duration_h
FROM
    yellow_trips_data
SORT BY trip_duration_h DESC
""").show()

+-------------+---------------+
|trip_distance|trip_duration_h|
+-------------+---------------+
|          0.0|            143|
|         0.96|             26|
|         1.91|             23|
|         1.93|             23|
|         2.52|             23|
|         5.32|             23|
|         0.73|             23|
|        18.58|             23|
|         1.49|             23|
|          0.7|             23|
|         2.21|             23|
|         3.37|             23|
|         1.95|             23|
|         1.06|             23|
|         3.29|             23|
|         4.57|             23|
|         1.55|             23|
|         0.82|             23|
|        14.24|             23|
|         1.98|             23|
+-------------+---------------+
only showing top 20 rows



# Longest Duration in hours

In [29]:
spark.sql("""
SELECT
    MAX(DATEDIFF(hour, tpep_pickup_datetime, tpep_dropoff_datetime)) AS max_date_diff
FROM
    yellow_trips_data
""").show()

[Stage 41:==============>                                           (1 + 3) / 4]

+-------------+
|max_date_diff|
+-------------+
|          162|
+-------------+



In [32]:
df_zones = spark.read.parquet('zones/')

In [33]:
df_zones.show()

+----------+-------------+--------------------+------------+
|LocationID|      Borough|                Zone|service_zone|
+----------+-------------+--------------------+------------+
|         1|          EWR|      Newark Airport|         EWR|
|         2|       Queens|         Jamaica Bay|   Boro Zone|
|         3|        Bronx|Allerton/Pelham G...|   Boro Zone|
|         4|    Manhattan|       Alphabet City| Yellow Zone|
|         5|Staten Island|       Arden Heights|   Boro Zone|
|         6|Staten Island|Arrochar/Fort Wad...|   Boro Zone|
|         7|       Queens|             Astoria|   Boro Zone|
|         8|       Queens|        Astoria Park|   Boro Zone|
|         9|       Queens|          Auburndale|   Boro Zone|
|        10|       Queens|        Baisley Park|   Boro Zone|
|        11|     Brooklyn|          Bath Beach|   Boro Zone|
|        12|    Manhattan|        Battery Park| Yellow Zone|
|        13|    Manhattan|   Battery Park City| Yellow Zone|
|        14|     Brookly

In [37]:
df_result = df_yellow.join(df_zones, df_yellow.PULocationID == df_zones.LocationID)

In [41]:
df_result.select('PULocationID', 'Zone').show()

+------------+--------------------+
|PULocationID|                Zone|
+------------+--------------------+
|         162|        Midtown East|
|          48|        Clinton East|
|         142| Lincoln Square East|
|         233| UN/Turtle Bay South|
|         262|      Yorkville East|
|         137|            Kips Bay|
|         142| Lincoln Square East|
|         230|Times Sq/Theatre ...|
|         142| Lincoln Square East|
|         162|        Midtown East|
|         229|Sutton Place/Turt...|
|         162|        Midtown East|
|         162|        Midtown East|
|         211|                SoHo|
|         142| Lincoln Square East|
|         161|      Midtown Center|
|         132|         JFK Airport|
|         239|Upper West Side S...|
|         239|Upper West Side S...|
|         247|      West Concourse|
+------------+--------------------+
only showing top 20 rows



In [43]:
df_result.createOrReplaceTempView('yellow_trips_with_zones')

In [52]:
spark.sql("""
SELECT
    Zone,
    COUNT(*) as nr_trips_from_zone
FROM
    yellow_trips_with_zones
GROUP BY Zone
ORDER BY nr_trips_from_zone ASC
""").show()

+--------------------+------------------+
|                Zone|nr_trips_from_zone|
+--------------------+------------------+
|Governor's Island...|                 1|
|       Rikers Island|                 2|
|       Arden Heights|                 2|
|         Jamaica Bay|                 3|
| Green-Wood Cemetery|                 3|
|Charleston/Totten...|                 4|
|   Rossville/Woodrow|                 4|
|       West Brighton|                 4|
|Eltingville/Annad...|                 4|
|       Port Richmond|                 4|
|         Great Kills|                 6|
|        Crotona Park|                 6|
|Heartland Village...|                 7|
|     Mariners Harbor|                 7|
|Saint George/New ...|                 9|
|             Oakwood|                 9|
|       Broad Channel|                10|
|New Dorp/Midland ...|                10|
|         Westerleigh|                12|
|     Pelham Bay Park|                12|
+--------------------+------------